This notebooks is created using Chapter 2 of the the [Advanced NLP with spaCy](https://course.spacy.io/en/chapter2) course

# Chapter 2: Large-scale data analysis with spaCy

## Data Structures

### Shared vocab and string store (1)

spaCy stores all shared data in a vocabulary, the Vocab.

This includes words, but also the labels schemes for tags and entities.

To save memory, all strings are encoded to hash IDs. If a word occurs more than once, we don't need to save it every time.

Instead, spaCy uses a hash function to generate an ID and stores the string only once in the string store. The string store is available as `nlp.vocab.strings`.

It's a lookup table that works in both directions. You can look up a string and get its hash, and look up a hash to get its string value. Internally, spaCy only communicates in hash IDs.

Hash IDs can't be reversed, though. If a word is not in the vocabulary, there's no way to get its string. That's why we always need to pass around the shared vocab.

- `Vocab`: stores data shared across multiple documents
- To save memory, spaCy encodes all strings to hash values
- Strings are only stored once in the `StringStore` via `nlp.vocab.strings`
- String store: **lookup** table in both directions

In [2]:
# Import spaCy
import spacy

# Create a blank English nlp object
nlp = spacy.blank("en")

In [3]:
nlp.vocab.strings.add("coffee")
coffee_hash = nlp.vocab.strings["coffee"]
coffee_string = nlp.vocab.strings[coffee_hash]

- Hashes can't be reversed – that's why we need to provide the shared vocab

In [8]:
coffee_hash

3197928453018144401

In [4]:
# Raises an error if we haven't seen the string before
string = nlp.vocab.strings[3197928453018144401]

In [6]:
string

'coffee'

In [7]:
nlp.vocab.strings

### Shared vocab and string store (2)

To get the hash for a string, we can look it up in `nlp.vocab.strings`.

To get the string representation of a hash, we can look up the hash.

A `Doc` object also exposes its vocab and strings.

- Look up the string and hash in `nlp.vocab.strings`

In [9]:
doc = nlp("I love coffee")
print("hash value:", nlp.vocab.strings["coffee"])
print("string value:", nlp.vocab.strings[3197928453018144401])

hash value: 3197928453018144401
string value: coffee


- The `doc` also exposes the vocab and strings

In [10]:
doc = nlp("I love coffee")
print("hash value:", doc.vocab.strings["coffee"])

hash value: 3197928453018144401


In [11]:
print("hash value:", doc.vocab.strings["love"])

hash value: 3702023516439754181


### Lexemes: entries in the vocabulary

Lexemes are context-independent entries in the vocabulary.

You can get a lexeme by looking up a string or a hash ID in the vocab.

Lexemes expose attributes, just like tokens.

They hold context-independent information about a word, like the text, or whether the word consists of alphabetic characters.

Lexemes don't have part-of-speech tags, dependencies or entity labels. Those depend on the context.

- A Lexeme object is an entry in the vocabulary

In [12]:
doc = nlp("I love coffee")
lexeme = nlp.vocab["coffee"]

# Print the lexical attributes
print(lexeme.text, lexeme.orth, lexeme.is_alpha)

coffee 3197928453018144401 True


- Contains the context-independent information about a word
  - Word text: lexeme.text and lexeme.orth (the hash)
  - Lexical attributes like lexeme.is_alpha
  - Not context-dependent part-of-speech tags, dependencies or entity labels

### Vocab, hashes and lexemes

Here's an example.

The Doc contains words in context – in this case, the tokens "I", "love" and "coffee" with their part-of-speech tags and dependencies.

Each token refers to a lexeme, which knows the word's hash ID. To get the string representation of the word, spaCy looks up the hash in the string store.

![Vocab, hashes and lexemes](./img/vocab_stringstore.png)


## Strings to hashes

Part 1
- Look up the string “cat” in `nlp.vocab.strings` to get the hash.
- Look up the hash to get back the string.

In [14]:
import spacy

nlp = spacy.blank("en")
doc = nlp("I have a cat")

# Look up the hash for the word "cat"
cat_hash = nlp.vocab.strings["cat"]
print(cat_hash)

# Look up the cat_hash to get the string
cat_string = nlp.vocab.strings[cat_hash]
print(cat_string)

5439657043933447811
cat


Part 2
- Look up the string label “PERSON” in `nlp.vocab.strings` to get the hash.
- Look up the hash to get back the string.

In [15]:
import spacy

nlp = spacy.blank("en")
doc = nlp("David Bowie is a PERSON")

# Look up the hash for the string label "PERSON"
person_hash = nlp.vocab.strings["PERSON"]
print(person_hash)

# Look up the person_hash to get the string
person_string = nlp.vocab.strings[person_hash]
print(person_string)

380
PERSON


## Vocab, hashes and lexemes

Why does this code throw an error?

In [19]:
import spacy

# Create an English and German nlp object
nlp = spacy.blank("en")
nlp_de = spacy.blank("de")

# Get the ID for the string 'Bowie'
bowie_id = nlp.vocab.strings["Bowie"]
print(bowie_id)

# Look up the ID for "Bowie" in the vocab
print(nlp_de.vocab.strings[bowie_id])

2644858412616767388


KeyError: "[E018] Can't retrieve string for hash '2644858412616767388'. This usually refers to an issue with the `Vocab` or `StringStore`."

The string "Bowie" isn’t in the German vocab, so the hash can’t be resolved in the string store.

Hashes can’t be reversed. To prevent this problem, add the word to the new vocab by processing a text or looking up the string, or use the same vocab to resolve the hash back to a string.

## Data Structures(2)

Now that you know all about the vocabulary and string store, we can take a look at the most important data structure: the `Doc`, and its views `Token` and `Span`.

### The Doc object

The `Doc` is one of the central data structures in spaCy. It's created automatically when you process a text with the `nlp` object. But you can also instantiate the class manually.

After creating the `nlp` object, we can import the `Doc` class from `spacy.tokens`.

Here we're creating a doc from three words. The spaces are a list of boolean values indicating whether the word is followed by a space. Every token includes that information – even the last one!

The `Doc` class takes three arguments: the shared vocab, the words and the spaces.

In [20]:
# Create an nlp object
import spacy
nlp = spacy.blank("en")

# Import the Doc class
from spacy.tokens import Doc

# The words and spaces to create the doc from
words = ["Hello", "world", "!"]
spaces = [True, False, False]

# Create a doc manually
doc = Doc(nlp.vocab, words=words, spaces=spaces)

### The Span object(1)

A `Span` is a slice of a doc consisting of one or more tokens. The `Span` takes at least three arguments: the doc it refers to, and the start and end index of the span. Remember that the end index is exclusive!

![The Span Object](./img/span_indices.png)

### The Span object(2)

To create a `Span` manually, we can also import the class from `spacy.tokens`. We can then instantiate it with the doc and the span's start and end index, and an optional label argument.

The `doc.ents` are writable, so we can add entities manually by overwriting it with a list of spans.

In [23]:
# Import the Doc and Span classes
from spacy.tokens import Doc, Span

# The words and spaces to create the doc from
words = ["Hello", "world", "!"]
spaces = [True, False, False]

# Create a doc manually
doc = Doc(nlp.vocab, words=words, spaces=spaces)

# Create a span manually
span = Span(doc, 0, 2)

# Create a span with a label
span_with_label = Span(doc, 0, 2, label="GREETING")

# Add span to the doc.ents
doc.ents = [span_with_label]

### Best practices

A few tips and tricks before we get started:

The `Doc` and `Span` are very powerful and optimized for performance. They give you access to all references and relationships of the words and sentences.

If your application needs to output strings, make sure to convert the doc as late as possible. If you do it too early, you'll lose all relationships between the tokens.

To keep things consistent, try to use built-in token attributes wherever possible. For example, `token.i` for the token index.

Also, don't forget to always pass in the shared vocab!

- `Doc` and `Span` are very powerful and hold references and relationships of words and sentences
  - **Convert result to strings as late as possible**
  - **Use token attributes if available** – for example, `token.i` for the token index
  
- Don't forget to pass in the shared `vocab`

## Creating a Doc

Let’s create some Doc objects from scratch!

Part 1

Import the `Doc` from `spacy.tokens`.

Create a `Doc` from the `words` and `spaces`. Don’t forget to pass in the vocab!

In [1]:
import spacy

nlp = spacy.blank("en")

# Import the Doc class
from spacy.tokens import Doc

# Desired text: "spaCy is cool!"
words = ["spaCy", "is", "cool", "!"]
spaces = [True, True, False, False]

# Create a Doc from the words and spaces
doc = Doc(nlp.vocab, words=words, spaces=spaces)
print(doc.text)

spaCy is cool!


Part 2

Import the `Doc` from `spacy.tokens`.

Create a `Doc` from the `words` and `spaces`. Don’t forget to pass in the vocab!

In [2]:
import spacy

nlp = spacy.blank("en")

# Import the Doc class
from spacy.tokens import Doc

# Desired text: "Go, get started!"
words = ["Go", ",", "get", "started", "!"]
spaces = [False, True, True, False, False]

# Create a Doc from the words and spaces
doc = Doc(nlp.vocab, words=words, spaces=spaces)
print(doc.text)

Go, get started!


Part 3

Import the `Doc` from `spacy.tokens`.

Complete the `words` and `spaces` to match the desired text and create a `doc`.

In [3]:
import spacy

nlp = spacy.blank("en")

# Import the Doc class
from spacy.tokens import Doc

# Desired text: "Oh, really?!"
words = ["Oh", ",", "really", "?", "!"]
spaces = [False, True, False, False, False]

# Create a Doc from the words and spaces
doc = Doc(nlp.vocab, words=words, spaces=spaces)
print(doc.text)

Oh, really?!
